In [1]:
import math 
from suffix import SuffixTree
from sys import getsizeof
from kmp import kmp
from collections import defaultdict

# Budowa DBF

Najbardziej wymagającą obliczeniowo częścią budowy DBF jest sortowanie. Teoretycznie najwydajniejsze jest wykorzystanie sortowania przez zliczanie, lecz w praktyce sortowanie funkcją biblioteczną `sorted` okazuje się być wydajniejsze.

## Implementacja sortowania przez zliczanie

Ostatecznie niewykorzystana

In [2]:
def countingSort(arr, exp1, key): 
    n = len(arr) 
    output = [0] * (n) 
    count = [0] * (10) 
    
    for i in range(0, n): 
        index = int(key(arr[i])/exp1) 
        count[ (index)%10 ] += 1
        
    for i in range(1,10): 
        count[i] += count[i-1] 
    i = n-1
    while i>=0: 
        index = int(key(arr[i])/exp1) 
        output[ count[ (index)%10 ] - 1] = arr[i] 
        count[ (index)%10 ] -= 1
        i -= 1
    i = 0
    for i in range(0,len(arr)): 
        arr[i] = output[i] 
        
def radixSort(arr, key): 
    max1 = max(map(key, arr))
    exp = 1
    while max1/exp > 0: 
        countingSort(arr, exp, key) 
        exp *= 10
    return arr

## Implementacja DBF

In [3]:
def sort_rename(sequence, letters=False):
    last_entry = None
    index = 0
    position_to_index = [None] * len(sequence)
    first_entry = {}

    enum_sequence = [(e, i) for i, e in enumerate(sequence)]
    sorted_entries = sorted(enum_sequence)
#     if letters: // linear sort; sorted() is faster
#         sorted_entries = radixSort(enum_sequence, lambda x: ord(x[0]))
#     else:
#         enum_sequence = radixSort(enum_sequence, lambda x: x[0][1])
#         sorted_entries = radixSort(enum_sequence,lambda x: x[0][0])

    for entry in sorted_entries:
        if(last_entry and last_entry[0] != entry[0]):
            index += 1
            first_entry[index] = entry[1]
        
        position_to_index[entry[1]] = index
        if last_entry is None:
            first_entry[0] = entry[1]
        last_entry = entry
    
    return(position_to_index,first_entry)

# Wyszukiwanie wzorca

## Stała długość wzorca

Po zbudowaniu DBF dla `pat&text` można bez problemu zmienić poszukiwany wzorzec na inny bez tworzenia całego DBF od nowa. Słownik zawierający (jako klucze) wszystkie napisy (o długości początkowego wzorca) występujące w tekście, może być użyty jako sposób na odnalezienie etykiety nowego wzorca. Takie rozwiązanie pozwoliłoby na znajdowanie przykładowego wystąpienia wzorca w stałym czasie, więc całe wyszukiwanie nadal miałoby złożoność O(n). Takie podejście wymaga jednak dodatkowej pamięci.  
Zamiast tego, można wykorzystać wyszukiwanie połówkowe do znalezienia wzorca w tablicy `pos`. Jako, że złożoność wyszukiwania połówkowego to O(log n) to całe wyszukiwanie nadal będzie O(n).  
Poniższa implementacja wyszukuje w tekście wzorzec o stałej długości opierając się na pojedynczym DBF. Jest ona tylko wstępem do lepszego wyszukiwania (opartego o algorytm kmr), więc wyszukuje w tablicy `pos` liniowo - wyszukiwanie połówkowe jest zaimplementowane w następnym algorytmie.

In [4]:
class dbf_search():
    def __init__(self, pat, text):
        assert len(pat) <= len(text), "pattern too long"
        pattext = pat + "&" +text
        self.k = len(pat)
        position_to_index, first_entry = sort_rename([pattext[i:i+self.k]
                                                      for i in range(len(pattext) - self.k+1)])
        self.name = position_to_index
        self.pos = first_entry
        self.pat = pat
        self.pattext = list(pattext)
        
    def find_pat(self, pat):
        pat = list(pat)
        for i in range(len(self.pos)):
            idx = self.pos[i]
            if pat == self.pattext[idx:idx+self.k]:
                return self.name[idx]
        return -1
        
    def search(self, pat):
        assert len(pat) == self.k
        pat_num = self.name[0]
        if self.pat != pat:
            pat_num = self.find_pat(pat)
            if pat_num == -1: # pattern not in text (not in `self.pos`)
                return []

        return [i-self.k-1 for i in range(self.k + 1, len(self.pattext) - self.k + 1) if self.name[i] == pat_num]

In [5]:
s = dbf_search("bb", "aabbaa")
print(s.search("bb"), s.search("aa"))

[2] [0, 4]


In [8]:
s = dbf_search("a", "xyzxyzxy")
print(s.search("xyz"))

AssertionError: 

## Zmienna długość wzorca

### Implementacja KMR i opartego na nim wyszukiwania wzorca

Aby zrealizować wyszukiwanie wzorca o dowolnej długości bez konieczności przebudowywania DBF należy wykorzystać algorytm kmr - wynik jego działania pozwoli na wyszukanie dowolnego wzorca po rozłożeniu go na podwzroce, których długość jest potęgą 2. Podobnie jak w powyższej implementacji, przed wyszukiwaniem wzorca w odpowiadającej mu tablicy `name` należy znaleźć przypisaną mu etykietę. Jest to zrealizowane przez wyszukiwanie połówkowe w tablicy `pos` - zwykle będzie to oznaczało konieczność wyszukania 2 etykiet (po 1 dla każdego fragmentu rozbitego wzorca). Jeżeli wzorca nie ma w tablicy `pos` to oznacza to, że nie występuje on w tekście.

In [7]:
def kmr(text):
    original_length = len(text)
    factor = math.floor(math.log2(len(text)))
    max_lenght = 2**factor
    padding_lenght = max_lenght - 1
    text += "z" * padding_lenght

    position_to_index, first_entry = sort_rename(list(text), letters=True)
    names={0: position_to_index}
    entries={0: first_entry}
    for i in range(1,factor+1):
        power = 2**(i-1)
        new_sequence = []
        for j in range(len(text)):
            if (j+power<len(names[i-1])):
                new_sequence.append((names[i-1][j], names[i-1][j+power]))
        position_to_index, first_entry = sort_rename(new_sequence)
        names[i] = position_to_index
        entries[i] = first_entry
    return(names,entries)

In [8]:
class kmr_search():
    def __init__(self, text):
        names, entries = kmr(text)
        factor = math.floor(math.log2(len(text)))
        padding_lenght = 2**factor - 1
        text += "z" * padding_lenght

        self.names = names
        self.entries = entries
        self.text = text
        
    def find_label(self, pattern, factor, size):
        "binary search for label of a pattern"
        left = 0
        right = len(self.entries[factor]) - 1
        mid = (right - left) // 2 + left
        idx = self.entries[factor][mid]
        while self.text[idx:idx+size] != pattern:
            if self.text[idx:idx+size] < pattern and len(self.text[idx:idx+size]) == size:
                left = mid
            else:
                right = mid
            mid = (right - left) // 2 + left
            if left == mid:
                idx = self.entries[factor][left]
                if self.text[idx:idx+size] == pattern:
                    break
                idx = self.entries[factor][right]
                if self.text[idx:idx+size] == pattern:
                    break
                return -1
                
            idx = self.entries[factor][mid]
        
        return self.names[factor][idx]
            
    def search(self, pat):
        factor = math.floor(math.log2(len(pat)))
        pat_size = 2**factor
        if pat_size == len(pat): # one pattern
            label = self.find_label(pat, factor, pat_size)
            return [i for i in range(len(self.text) - pat_size + 1) if self.names[factor][i] == label]
        else: # two patterns 
            patterns =[pat[:pat_size], pat[-pat_size:]]
            shift = len(pat) - pat_size
            labels = [self.find_label(pattern, factor, pat_size) for pattern in patterns]
         #   print(patterns, shift, labels)
            return [i for i in range(len(self.text) - pat_size + 1) 
                    if self.names[factor][i] == labels[0] and
                       self.names[factor][i+shift] == labels[1]]
            

In [9]:
s = kmr_search("aabbaa")
print(s.search("bb"), s.search("aa"), s.search("abb"))

[2] [0, 4] [1]


# Porównanie czasu budowy

In [15]:
texts = []
for filename in ["1997_714.txt", "romeo-i-julia-700.txt", "zad6"]:
    with open(filename, "r") as f:
        texts.append(f.read() + "$")

## Ustawa

In [16]:
%%timeit
SuffixTree(texts[0])

2.12 s ± 62.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
kmr_search(texts[0])

16.9 s ± 323 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Romeo i Julia

In [18]:
%%timeit
SuffixTree(texts[1])

88 ms ± 805 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit
kmr_search(texts[1])

404 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## zad6

In [21]:
%%timeit
SuffixTree(texts[2])

7.04 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%%timeit
kmr_search(texts[2])

16.3 ms ± 3.73 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Budowa drzewa suffixów zajmuje mniej czasu niż budowanie DBF.

# Porównanie rozmiarów

Praktycznie cała pamięć wymagana do wyszukiwania jest potrzebna do przetrzymywania tablic `name` i `pos`, więc właśnie ich wielkość zostanie zmierzona.

In [22]:
texts = []
for filename in ["1997_714.txt", "romeo-i-julia-700.txt", "zad6"]:
    with open(filename, "r") as f:
        texts.append((f.read() + "$", filename))

In [28]:
for text, filename in texts:
    searcher = kmr_search(text)
    size = sum([getsizeof(k) + getsizeof(v) for k, v in searcher.entries.items()])
    size += sum([getsizeof(k) + getsizeof(v) for  k, v  in searcher.names.items()])
    print(filename)
    print("rozmiar tekstu: ", getsizeof(text), "b   |   rozmiar potrzebnych struktur: ", size,"b")
    print(f"jest to około {size /getsizeof(text):.0f}x więcej\n" + "-" * 30)

1997_714.txt
rozmiar tekstu:  493020 b   |   rozmiar potrzebnych struktur:  197120696 b
jest to około 400x więcej
------------------------------
romeo-i-julia-700.txt
rozmiar tekstu:  25376 b   |   rozmiar potrzebnych struktur:  9028488 b
jest to około 356x więcej
------------------------------
zad6
rozmiar tekstu:  1886 b   |   rozmiar potrzebnych struktur:  423000 b
jest to około 224x więcej
------------------------------


Potrzebne struktury wymagają znacznych ilości pamięci w porównaniu z wielkością tesktu.

# Porównanie czasu wyszukiwania z algorytmem kmp

In [30]:
text = texts[0][0] # ustawa
searcher = kmr_search(text)

In [38]:
%%timeit
searcher.search("a")

33 ms ± 697 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
%%timeit
kmp(text, "a")

42.1 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
%%timeit
searcher.search("przyczyny")

46 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
%%timeit
kmp(text, "przyczyny")

39 ms ± 1.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
%%timeit
searcher.search("również takich, które zostały przez sprzedawcę zapakowane lub rozważone")

53.8 ms ± 3.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
%%timeit
kmp(text, "również takich, które zostały przez sprzedawcę zapakowane lub rozważone")

41.1 ms ± 4.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Czasy wyszukiwania są zbliżone - gdyby wykorzystać wyszukiwanie etykiet w tablicy `pos` przy użyciu słownika zapewnie szybsze byłoby wyszukiwanie oparte na DBF